In [1]:
def polynomialFeatures(X, degree):
    return

In [2]:
def mse(Y_true, Y_pred):
    return

In [3]:
def learning_curve(model, X, Y, cv, train_size=1, learning_rate=0.01, epochs=1000, tol=None, regularizer=None, lambd=0.0, **kwargs):
    return

In [4]:
def plot_polynomial_model_complexity(model, X, Y, cv, maxPolynomialDegree, learning_rate=0.01, epochs=1000, tol=None, regularized=None, lambd=0.0, **kwargs):
    return

In [ ]:
class Linear_Regression:
    def __init__(self):
        pass
    
    def fit(self, X, Y, learning_rate=0.01, epochs=1000, tol=None, regularizer=None, lambd=0.0, **kwargs):
        return
    
    def predict(self, X):
        return